# 7.3  다나와 크롤링 데이터 전처리

### 7.3.1 다나와 크롤링 데이터 불러오기 

In [ ]:
# 예제 7-17 다나와 크롤링 결과 가져오기 
import pandas as pd

data = pd.read_excel('./files/1_danawa_crawling_result.xlsx')
data.info()
data.head()

### 7.3.2 회사명, 모델명 정리 

In [ ]:
# 예제 7-18 회사명 + 모델명 분리
company_list = []
product_list = []
for title in data['상품명']:
    title_info = title.split(' ', 1)
    company_name = title_info[0]
    product_name = title_info[1]
    company_list.append(company_name)
    product_list.append(product_name)

### 7.3.4 스펙 목록에서 카테고리, 사용시간, 흡입력을 추출해서 정리 


In [ ]:
# 예제 7-19 첫 번째 제품의 스펙 목록 분리
spec_list = data['스펙 목록'][0].split(' / ')
spec_list 

In [ ]:
# 예제 7-20 카테고리 정보 추출
category = spec_list[0] 
category 

In [ ]:
# 예제 7-21 ‘사용시간’, ‘흡입력’이 포함된 원소 추출 
for spec in spec_list:
    if '사용시간' in spec: 
         use_time_spec = spec 
    elif '흡입력' in spec:
        suction_spec = spec  
print(use_time_spec)
print(suction_spec)

In [ ]:
# 예제 7-22 정량적인 수치 추출 
use_time_value = use_time_spec.split(' ')[1].strip() 
suction_value = suction_spec.split(' ')[1].strip()
print(use_time_value)
print(suction_value)

In [ ]:
# 예제 7-23 카테고리, 사용시간, 흡입력 추출 
category_list = []
use_time_list = []
suction_list = [] 
for spec_data in data['스펙 목록']:
    # ' / ' 기준으로 스펙 분리하기 
    spec_list = spec_data.split(' / ')
    
    # 카테고리 추출하기
    category = spec_list[0] 
    category_list.append(category)
    
    # 사용시간, 흡입력 추출 
    ## 사용시간, 흡입력 정보가 없는 제품을 위해 변수를 생성 
    use_time_value = None 
    suction_value = None 
    
    ## spec_list의 각 원소에서 사용시간, 흡입력 수치 추출
    for spec in spec_list:
        if '사용시간' in spec: 
            use_time_value = spec.split(' ')[1].strip()             
        if '흡입력' in spec:
            suction_value = spec.split(' ')[1].strip() 
    use_time_list.append(use_time_value)
    suction_list.append(suction_value)

In [ ]:
# 예제 7-24 카테고리, 사용시간, 흡입력에 대한 전처리 결과 확인
print("카테고리", len(category_list), category_list[0:5])
print("사용시간", len(use_time_list), use_time_list[0:5])
print("흡입력", len(suction_list), suction_list[0:5])  # 수정(2022.05.19)


### 7.3.5 무선청소기 사용시간 단위 통일시키기

In [ ]:
# 예제 7-25 사용시간을 분 단위로 조정하는 함수
def convert_time_minute(time):
    try:
        if '시간' in time:
            hour = time.split('시간')[0]
            if '분' in time:
                minute = time.split('시간')[-1].split('분')[0]
            else:
                minute = 0
        else: 
            hour = 0
            minute = time.split('분')[0]
        return int(hour)*60 + int(minute)
    except:
        return None                

In [ ]:
# 예제 7-26 사용시간을 분 단위로 조정하는 함수의 테스트
times = ["40분", "4분", "1시간", "3시간30분", "4시간"]
for time in times:
    time_value = convert_time_minute(time)
    print(time, "=", time_value)

In [ ]:
# 예제 7-27 모델별 사용시간을 분 단위로 통일하기
new_use_time_list = []
for time in use_time_list:
    value = convert_time_minute(time)
    new_use_time_list.append(value)
    
print(len(new_use_time_list))
print(new_use_time_list[0:10])

### 7.3.6 무선 청소기 흡입력 단위 통일시키기

In [ ]:
# 예제 7-28 흡입력 단위를 통일시키는 함수
def get_suction(value):
    try:
        value = value.upper()
        if "AW" in value or "W" in value:
            result = value.replace("A", "").replace("W","")
            result = int(result.replace(",",""))
        elif "PA" in value:
            result = value.replace("PA","")
            result = int(result.replace(",",""))/100
        else:
            result = None
        return result
    except:
        return None

In [ ]:
# 예제 7-29 흡입력 단위 통일시키기
new_suction_list = []
for power in suction_list:
    value = get_suction (power)
    new_suction_list.append(value)

print(len(new_suction_list))
print(new_suction_list[0:10])

### 7.3.7 다나와 전처리 결과를 엑셀로 저장


In [ ]:
# 예제 7-30 전처리 데이터 확인
pd_data = pd.DataFrame()
pd_data['카테고리'] = category_list
pd_data['회사명'] = company_list
pd_data['제품'] = product_list
pd_data['가격'] = data['가격']
pd_data['사용시간'] = new_use_time_list
pd_data['흡입력'] = new_suction_list
pd_data.head()

In [ ]:
# 예제 7-31 카테고리 분류 기준 및 데이터 개수 점검
pd_data['카테고리'].value_counts() 

In [ ]:
# 예제 7-32 핸디/스틱청소기만 선택
pd_data_final = pd_data[pd_data['카테고리'].isin(['핸디/스틱청소기'])]
len(pd_data_final)

In [ ]:
# 추가.  가격 데이터 살펴보기 2022.05.19  
pd_data_final['가격'].unique()   # 이상값 확인  "가격비교예정" 제외 필요

In [ ]:
# 추가. "가격비교예정"  상품 제외하기 2022.05.19
cond = pd_data_final['가격'] != '가격비교예정'
pd_data_final = pd_data_final[cond]
pd_data_final

In [ ]:
# 예제 7-33 엑셀로 저장
pd_data_final.to_excel('./files/2_danawa_data_final.xlsx', index = False)